In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, RepeatVector
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

file_path = './dog.txt'
data = pd.read_csv(file_path, sep='\t')

# Pad sequences to the same length
min_length = min(data['sequence'].apply(len))   #we can try with max(it might probably give better results since using min, we're not considering all the repeat patterns in the sequence that might be responsible for making it to be in a particular cluster )
def pad_sequence(sequence, min_length):         #I used min to make the code run faster as max seemed to take hours.
    if len(sequence) >= min_length:
        padded_sequence = sequence[:min_length]
    else:
        padding = "N" * (min_length - len(sequence))
        padded_sequence = sequence + padding
    return padded_sequence

data['padded_sequence'] = data['sequence'].apply(lambda x: pad_sequence(x, min_length))

# One-hot encode sequences
def one_hot_encode(sequence):
    encoder = OneHotEncoder(categories=[['A', 'C', 'G', 'T', 'N']])
    sequence_array = np.array(list(sequence)).reshape(-1, 1)
    encoded = encoder.fit_transform(sequence_array).toarray()
    return encoded

data['encoded_sequence'] = data['padded_sequence'].apply(lambda x: one_hot_encode(x))

X = np.stack(data['encoded_sequence'].to_numpy(), axis=0)

# Split the encoded sequences into training and testing sets
X_train, X_test = train_test_split(X, test_size=0.3, random_state=42)

# Define autoencoder architecture
input_seq = Input(shape=(X_train[0].shape))
encoded = LSTM(64)(input_seq)
decoded = RepeatVector(X_train.shape[1])(encoded)
decoded = LSTM(5, return_sequences=True)(decoded)

# Create autoencoder model
autoencoder = Model(input_seq, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=200, batch_size=32, validation_split=0.2, verbose=1)

# Extract the encoder part of the autoencoder to obtain the learned representations
encoder = Model(input_seq, encoded)
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)

# Cluster the sequences using K-means
n_clusters = 7
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
train_clusters = kmeans.fit_predict(X_train_encoded)
test_clusters = kmeans.predict(X_test_encoded)

# Evaluate the clustering performance using silhouette score
train_silhouette = silhouette_score(X_train_encoded, train_clusters)
test_silhouette = silhouette_score(X_test_encoded, test_clusters)

print(f"Train silhouette score: {train_silhouette:.4f}")
print(f"Test silhouette score: {test_silhouette:.4f}")


Epoch 1/200
15/15 [==============================] - 38s 566ms/step - loss: 0.1660 - val_loss: 0.1509
Epoch 2/200
15/15 [==============================] - 4s 278ms/step - loss: 0.1501 - val_loss: 0.1492
Epoch 3/200
15/15 [==============================] - 4s 286ms/step - loss: 0.1491 - val_loss: 0.1486
Epoch 4/200
15/15 [==============================] - 4s 293ms/step - loss: 0.1486 - val_loss: 0.1483
Epoch 5/200
15/15 [==============================] - 4s 255ms/step - loss: 0.1484 - val_loss: 0.1481
Epoch 6/200
15/15 [==============================] - 4s 283ms/step - loss: 0.1482 - val_loss: 0.1479
Epoch 7/200
15/15 [==============================] - 4s 286ms/step - loss: 0.1480 - val_loss: 0.1478
Epoch 8/200
15/15 [==============================] - 4s 287ms/step - loss: 0.1478 - val_loss: 0.1474
Epoch 9/200
15/15 [==============================] - 5s 331ms/step - loss: 0.1476 - val_loss: 0.1471
Epoch 10/200
15/15 [==============================] - 4s 289ms/step - loss: 0.1473 - val_l

15/15 [==============================] - 2s 155ms/step - loss: 0.1453 - val_loss: 0.1449
Epoch 82/200
15/15 [==============================] - 2s 153ms/step - loss: 0.1453 - val_loss: 0.1450
Epoch 83/200
15/15 [==============================] - 2s 158ms/step - loss: 0.1454 - val_loss: 0.1450
Epoch 84/200
15/15 [==============================] - 2s 150ms/step - loss: 0.1453 - val_loss: 0.1450
Epoch 85/200
15/15 [==============================] - 2s 150ms/step - loss: 0.1454 - val_loss: 0.1450
Epoch 86/200
15/15 [==============================] - 2s 149ms/step - loss: 0.1454 - val_loss: 0.1449
Epoch 87/200
15/15 [==============================] - 2s 154ms/step - loss: 0.1453 - val_loss: 0.1449
Epoch 88/200
15/15 [==============================] - 2s 161ms/step - loss: 0.1454 - val_loss: 0.1449
Epoch 89/200
15/15 [==============================] - 2s 152ms/step - loss: 0.1453 - val_loss: 0.1449
Epoch 90/200
15/15 [==============================] - 2s 153ms/step - loss: 0.1453 - val_loss: 

Epoch 161/200
15/15 [==============================] - 2s 155ms/step - loss: 0.1450 - val_loss: 0.1446
Epoch 162/200
15/15 [==============================] - 2s 155ms/step - loss: 0.1450 - val_loss: 0.1447
Epoch 163/200
15/15 [==============================] - 3s 178ms/step - loss: 0.1450 - val_loss: 0.1446
Epoch 164/200
15/15 [==============================] - 2s 154ms/step - loss: 0.1450 - val_loss: 0.1447
Epoch 165/200
15/15 [==============================] - 2s 152ms/step - loss: 0.1450 - val_loss: 0.1446
Epoch 166/200
15/15 [==============================] - 2s 151ms/step - loss: 0.1450 - val_loss: 0.1447
Epoch 167/200
15/15 [==============================] - 2s 153ms/step - loss: 0.1450 - val_loss: 0.1447
Epoch 168/200
15/15 [==============================] - 2s 151ms/step - loss: 0.1450 - val_loss: 0.1447
Epoch 169/200
15/15 [==============================] - 2s 151ms/step - loss: 0.1451 - val_loss: 0.1446
Epoch 170/200
15/15 [==============================] - 2s 149ms/step - lo